In [ ]:
import pickle
import numpy as np
import pandas as pd
import shap
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, space_eval
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler

In [ ]:
# 0) Reproducibility

random_seed = 42
np.random.seed(random_seed)

In [ ]:
# 1) Paths (edit as needed)

model_path_in  = r"C:/Users/hangang/Desktop/best_xgb_model_full.pkl"
data_path      = r"C:/Users/hangang/Desktop/01_data_full.csv"
model_path_out = r"C:/Users/hangang/Desktop/best_XGB_model_topk.pkl"
log_path_out   = r"C:/Users/hangang/Desktop/topk_search_log.csv"

In [ ]:
# 2) Load saved artifact (expects dict with model, scaler, features)
# =========================
with open(model_path_in, "rb") as f:
    obj = pickle.load(f)

if not (isinstance(obj, dict) and "model" in obj):
    raise ValueError("Saved artifact must be a dict with keys {'model','scaler','features'}.")

base_model    = obj["model"]        # only used to keep consistency; we will retrain new models per K
saved_scaler  = obj.get("scaler", None)
saved_features= obj.get("features", None)

if saved_scaler is None:
    raise ValueError("No scaler found in the saved artifact.")

# scaler class to refit per loop (avoid leakage)
scaler_cls = saved_scaler.__class__

In [ ]:
# 3) Load data and split
# =========================
df = pd.read_csv(data_path, encoding="utf-8")
if "Chl-a" not in df.columns:
    raise KeyError("'Chl-a' column not found in the CSV.")

X_all = df.drop("Chl-a", axis=1)
y_all = df["Chl-a"]

# align to saved feature order if present
if saved_features is not None:
    missing = [c for c in saved_features if c not in X_all.columns]
    if missing:
        raise ValueError(f"Missing features in CSV: {missing}")
    X_all = X_all[saved_features]

X_train, X_test, y_train, y_test = train_test_split(
    X_all, y_all, test_size=0.3, random_state=random_seed
)

In [ ]:
# 4) Helper: metrics and hyperopt space (same ranges as before)
# =========================
def metrics(y_true, y_pred):
    r2   = r2_score(y_true, y_pred)
    rmse = float(np.sqrt(mean_squared_error(y_true, y_pred)))
    mae  = float(mean_absolute_error(y_true, y_pred))
    return r2, rmse, mae

space = {
    'n_estimators':     hp.quniform('n_estimators', 10, 80, 1),
    'max_depth':        hp.quniform('max_depth', 1, 7, 1),
    'learning_rate':    hp.uniform('learning_rate', 0.01, 0.08),
    'gamma':            hp.uniform('gamma', 0.0, 6.0),
    'min_child_weight': hp.quniform('min_child_weight', 1, 20, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.15, 1.0),
    'alpha':            hp.uniform('alpha', 0.0, 6.0),
    'lambda':           hp.uniform('lambda', 0.0, 6.0),
}

def build_xgb(params):
    return xgb.XGBRegressor(
        n_estimators     = int(params['n_estimators']),
        max_depth        = int(params['max_depth']),
        learning_rate    = float(params['learning_rate']),
        gamma            = float(params['gamma']),
        min_child_weight = int(params['min_child_weight']),
        colsample_bytree = float(params['colsample_bytree']),
        reg_alpha        = float(params['alpha']),
        reg_lambda       = float(params['lambda']),
        random_state     = random_seed,
        n_jobs           = -1
    )

In [ ]:
# 5) SHAP ranking on TRAIN set (scaled with saved scaler class)
# =========================
scaler_for_rank = scaler_cls()
Xtr_scaled_for_rank = scaler_for_rank.fit_transform(X_train)

explainer = shap.TreeExplainer(base_model)
# If base_model was trained on scaled data, ranking should also use scaled features
shap_vals_train = explainer.shap_values(Xtr_scaled_for_rank, check_additivity=False)
mean_abs = np.mean(np.abs(shap_vals_train), axis=0)
rank_idx = np.argsort(-mean_abs)  # descending
ranked_features = X_train.columns.to_numpy()[rank_idx].tolist()

In [ ]:
# 6) Iterate K = 5..len(features), train/eval, pick best by R2 on TEST
# =========================
results = []
best_record = None  # (k, r2_test, rmse_test, mae_test, params, model, fitted_scaler, features_k)

for k in range(5, len(ranked_features) + 1):
    feats_k = ranked_features[:k]

    # fit scaler on TRAIN (subset columns) to avoid leakage
    scaler_k = scaler_cls()
    Xtr_k = scaler_k.fit_transform(X_train[feats_k])
    Xte_k = scaler_k.transform(X_test[feats_k])

    # hyperopt objective (3-fold CV on TRAIN)
    def objective(p):
        model_k = build_xgb(p)
        kf = KFold(n_splits=3, shuffle=True, random_state=random_seed)
        cv_scores = cross_val_score(model_k, Xtr_k, y_train, cv=kf, scoring='neg_mean_squared_error')
        mean_mse = -float(np.mean(cv_scores))
        return {'loss': mean_mse, 'status': STATUS_OK}

    trials = Trials()
    best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=200, trials=trials, rstate=np.random.default_rng(random_seed))
    best_params = space_eval(space, best)

    # train on full TRAIN with best params
    model_k = build_xgb(best_params)
    model_k.fit(Xtr_k, y_train)

    # evaluate on TEST
    yhat_te = model_k.predict(Xte_k)
    r2_te, rmse_te, mae_te = metrics(y_test, yhat_te)

    results.append({
        'k': k,
        'r2_test': r2_te,
        'rmse_test': rmse_te,
        'mae_test': mae_te,
        'best_params': best_params
    })

    # keep best by R2, tie-breaker by lower RMSE
    if (best_record is None) or (r2_te > best_record[1]) or (np.isclose(r2_te, best_record[1]) and rmse_te < best_record[2]):
        best_record = (k, r2_te, rmse_te, mae_te, best_params, model_k, scaler_k, feats_k)

# save search log
pd.DataFrame(results).to_csv(log_path_out, index=False)

In [ ]:
# 7) Save best model (with scaler & selected features)
# =========================
best_k, best_r2, best_rmse, best_mae, best_params, best_model, best_scaler, best_feats = best_record

with open(model_path_out, "wb") as f:
    pickle.dump({
        'model': best_model,
        'scaler': best_scaler,
        'features': best_feats,
        'random_seed': random_seed,
        'best_params': best_params,
        'ranked_features_all': ranked_features,
        'best_k': best_k,
        'cv_space': 'XGB ranges: n_estimators[10,80], max_depth[1,7], lr[0.01,0.08], gamma[0,6], min_child_weight[1,20], colsample_bytree[0.15,1], alpha[0,6], lambda[0,6]'
    }, f)

print(f"[Best K] {best_k}")
print(f"[Test R2/RMSE/MAE] {best_r2:.4f} / {best_rmse:.4f} / {best_mae:.4f}")
print(f"[Saved] {model_path_out}")
print(f"[Search log] {log_path_out}")